# Developer's Guide

`funflow` provides a collection of tasks to run most pipelines.
However, its strength is to allow extending this collection.

This tutorial aims to help prospective `funflow` developers get started with creating new tasks to be used by other `funflow` users.

## 1. Creating your own task

In this tutorial, we will create a task called `CustomTask` by defining its type.

We will define our own flow type, and write the necessary functions to be able to run it.

### Defining the new task

To define a task for our users, we first have to define a type that represents the task.

> A task is represented by a GADT datatype of kind `* -> * -> *`.

In [1]:
-- Required language extensions
{-# LANGUAGE GADTs, StandaloneDeriving #-}

-- Define the representation of a custom task with some String and Int parameters
data CustomTask i o where
    CustomTask :: String -> Int -> CustomTask String String

-- Necessary in order to display it
deriving instance (Show i, Show o) => Show (CustomTask i o)

Here, we create a type `SomeCustomTask` of type constructor `SomeCustomTask i o` and a value constructor `SomeCustomTask` of type `String -> Int -> SomeCustomTask String String`.

`String -> Int -> SomeCustomTask String String` means thatby providing a `String` and an `Int`, the function will give a task that take a `String` as input and produce a `String` as output.

A new task can be created by using the value constructor:

In [2]:
-- An example of instanciation
CustomTask "someText" 42

CustomTask "someText" 42

However, a value created as such cannot be used by itself to define flows: it lacks all the "properties" of flows.

For a task to be usable in a flow, we need some more work.

### From a task to a flow

The `Flow` type is restricted to the set of tasks defined by `RequiredStrands` in `Funflow.Flow`.

In order to manipulate a flow that can run our custom task, we need to create our own new type using `ExtendedFlow` from `Funflow.Flow`:

In [3]:
{-# LANGUAGE DataKinds, RankNTypes #-}
import Funflow.Flow (ExtendedFlow)

type MyFlow input output = ExtendedFlow '[ '("custom", CustomTask) ] input output

> The list and tuple writing preceeded by an apostrophe `'[ ... ]` and `'( ... )` are type-level lists and tuples.
>
> Basically, we define a type-level list of type-level tuples `(label, task type)` that associate a string label to a task type definition.
> 
> In `kernmantle`, such a tuple is called a _strand_.

Now that we have our own type of flow that uses our custom task, we can define how a value of our custom task should be _stranded_:

In [4]:
{-# LANGUAGE OverloadedLabels #-}
import Control.Kernmantle.Rope (strand)

someCustomFlow :: String -> Int -> MyFlow String String
someCustomFlow x y = strand #custom (CustomTask x y)

This function is called a _smart constructor_.
It facilitates the creation of a flow for a user without having to think about strands.

The `#custom` value is a Haskell label, and must match the string label associated to our task type in the flow type definition (here `"custom"`).

In [5]:
myFlow :: MyFlow String String
myFlow = someCustomFlow "woop!" 7

### Interpret a task

The strenght of `funflow` comes from its ability from distinguishing the representation of computations (tasks) from the actual implementation that they provoke.

We have defined a task, which is a representation of a computation.
We now need to implement said computation.

In order to do that, we write what is called an _interpreter function_.

An interpreter function is executed before running the flow.
It takes a value of the task type that matches a given _strand_ (identified by its label) and consumes it, in order to produce the representation of an actual computation.

In our case, we could define that our custom task `CustomTask n x` appends `n` times the string `x` to the input (which is a `String`).

In [6]:
import Control.Arrow (Arrow, arr)

-- Helper function that repeats a string n times
duplicate :: String -> Int -> String
duplicate s n = concat (replicate n s)

-- Our interpreter
interpretCustomTask :: (Arrow a) => CustomTask i o -> a i o
interpretCustomTask customTask = case customTask of
    CustomTask s n -> arr (\input -> input ++ duplicate s n)

What happens here is:

1. We get the `customTask` of our type `CustomTask`.
2. We differentiate the possible values.
   Here there is only one value constructor, but GADTs can hold multiple value constructors.
3. Since our function is pure, we can simply wrap it inside of an `Arrow` using `arr`.

`\input -> input ++ duplicate s n` is the actual function that will be executed when running the pipeline.

> In funflow, pure computation should be wrapped in a `Arrow` while IO operations should wrapped in a `Kleisli IO`.
> 
> Wrapping in an `Arrow` is done by using `arr`, while wrapping in a `Kleisli IO` is done by using `liftKleisliIO`.

`funflow`'s interpreter functions are defined in the `Funflow.Run` module and can serve as examples.

### Run your custom flow

Now that we have defined a way to run our task, we might as well run our pipeline!

When using only `funflow`'s task, there is a convenient function `runFlow` defined in `Funflow.Run`.
This function consumes a value of type `Flow`, which can be constituted of `funflow`'s tasks only.

The most simple way is to convert our custom flow type to the `funflow`'s flow type in order to use this function.

> In `kernmantle`, intepreting a task with a function is called _weaving_ a strand.
>
> There are multiple function available to weave strands (`weave`, `weave'`, `weave''`, `weaveK`).
> Almost always, the one you should be using is `weave'`.

In [7]:
import Control.Kernmantle.Rope ((&), weave')
import Funflow.Flow (Flow)

weaveMyFlow myFlow = myFlow & weave' #custom interpretCustomTask

> `kernmantle`'s `&` operator allows to weave in chain multiple strands:
> ```haskell
> weaveMyFlow myFlow = myFlow & weave' #custom1 interpretCustomTask1 & weave' #custom2 interpretCustomTask2
> ```

Now, we can run the resulting flow:

In [8]:
{-# LANGUAGE FlexibleContexts, ScopedTypeVariables #-}
import Control.Kernmantle.Rope (LooseRopeWith)
import Funflow.Flow (RequiredStrands, RequiredCore)
import Funflow.Run (runFlow)

runMyFlow :: MyFlow i o -> i -> IO o
runMyFlow myFlow input = runFlow (weaveMyFlow myFlow) input

Line 7: Eta reduce
Found:
runMyFlow myFlow input = runFlow (weaveMyFlow myFlow) input
Why not:
runMyFlow myFlow = runFlow (weaveMyFlow myFlow)

In [9]:
runMyFlow myFlow "Kangaroo goes " :: IO String

"Kangaroo goes woop!woop!woop!woop!woop!woop!woop!"

> We have to specify the type of the result `IO String` because of some issues with type inference when using GADTs.

## Going further

See more about `kernmantle` here: https://github.com/tweag/kernmantle